In [1]:
%load_ext autoreload
%autoreload 2
import adanet

In [2]:
OUTPUT_DIR = './tmp/adanet'
datadir = './data'
is_tensorboard = True

In [3]:
# this will kill the processes for Tensorboard
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill

# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill

kill: (4556)kill: (4562)

In [4]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)
print(x_train.shape, x_test.shape)

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414, 3) (47, 3)
(414, 224, 672) (47, 224, 672) (414, 3) (47, 3)
(414, 672, 224) (47, 672, 224) (414, 3) (47, 3)
(414, 672, 224) (47, 672, 224)


In [5]:
import p8_util_config
p8_util_config.ADANET_MAX_ITERATION_STEPS

20

In [9]:
import p8_util
import p8_util_config
import NNGenerator

dict_nn_layer_config = p8_util_config.dict_adanet_config['adanet_nn_layer_config']
nn_type = dict_nn_layer_config['nn_type']


nClasses = dict_nn_layer_config['nn_logit_dimension']
print("\n*** Number of classes= {} / NN type= {}".format(nClasses, nn_type))

my_config = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)

adanet_estimator = adanet.Estimator(
    head=p8_util_config.dict_adanet_config['adanet_tf_head'],
    
    subnetwork_generator=NNGenerator.NNGenerator(p8_util_config.dict_adanet_config),
            
    adanet_lambda=p8_util_config.dict_adanet_config['adanet_lambda'],#0.0,#.015,
    
    max_iteration_steps=p8_util_config.ADANET_MAX_ITERATION_STEPS,
    
    evaluator=adanet.Evaluator(
        input_fn=p8_util.input_fn("train", x_train, y_train\
                          , p8_util_config.NUM_EPOCHS\
                          , batch_size=p8_util_config.BATCH_SIZE\
                          , feature_shape = tuple_dimension\
                          ),
        #input_fn=p8_util.input_fn("train", x_train, y_train\
        #                  , p8_util_config.NUM_EPOCHS
        #                  , tuple_dimension=tuple_dimension\
        #                  , batch_size=p8_util_config.BATCH_SIZE),
        steps=None),
    config=  my_config)


*** Number of classes= 3 / NN type= CNN

*** make_config() : output dir= ./tmp/adanet/CNN
*** INFO : MyGenerator : instantiation DONE!
INFO:tensorflow:Using config: {'_model_dir': './tmp/adanet/CNN', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1d10097dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
import p8_util_config
print("Global steps= {}".format(p8_util_config.TRAIN_STEPS))
print("MAX Adanet steps= {}".format(p8_util_config.ADANET_MAX_ITERATION_STEPS))
y_train.shape

Global steps= 200
MAX Adanet steps= 20


(414, 3)

In [11]:
import time
import tensorflow as tf

start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    adanet_estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn= p8_util.input_fn("train"\
                           , x_train\
                           , y_train\
                           , p8_util_config.NUM_EPOCHS \
                           , feature_shape = tuple_dimension\
                           , batch_size=p8_util_config.BATCH_SIZE),
        max_steps=p8_util_config.TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn= p8_util.input_fn("test"
                                   , x_test
                                   , y_test
                                   , p8_util_config.NUM_EPOCHS\
                                   , feature_shape = tuple_dimension\
                                   , batch_size=p8_util_config.BATCH_SIZE),
        steps=None,
        start_delay_secs=1,
        throttle_secs=1,  
    )
)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.
INFO:tensorflow:Beginning training AdaNet iteration 0

*** input_fn() : feature_shape= (672, 224) / Label shape= [3]
INFO:tensorflow:Calling model_fn.

*** NNGenerator : layers= (0,1)

*** NNAdaNetBuilder : NN Type=CNN

*** NNAdaNetBuilder : NN Type=CNN
INFO:tensorflow:Building iteration 0
INFO:tensorflow:Building subnetwork 'CNN_linear'


*** build_subnetwork() : NN type= CNN / Input layer shape= (?, 150528)


TypeError: Tensors in list passed to 'values' of 'ConcatV2' Op have types [int32, <NOT CONVERTIBLE TO TENSOR>] that don't all match.

In [ ]:
int('CNN_layer_2'.split('_')[2])

In [ ]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("\n{}: {}".format(title, results[key]))


In [ ]:
#import tensorflow as tf
#help(tf.estimator.Estimator)

In [ ]:
def ensemble_architecture(result):
  """Extracts the ensemble architecture from evaluation results."""

  architecture = result["architecture/adanet/ensembles"]
  # The architecture is a serialized Summary proto for TensorBoard.
  summary_proto = tf.summary.Summary.FromString(architecture)
  return summary_proto.value[0].tensor.string_val[0]

In [ ]:
print("Architecture: {} / Accuracy= {} /Loss= {}".format(ensemble_architecture(results)\
                                                  , results['accuracy'], results['loss']))

In [ ]:
# Architecture: b'| DNN_layer_1 | DNN_layer_2 | DNN_layer_3 |' : accuracy = 40% / Loss = 1.0883579
# Architecture: b'| CNNBase_layer_1 |' / Accuracy= 0.40425533056259155 /Loss= 1.081491470336914
# Architecture: b'| CNNBase_linear | CNNBase_layer_1 |' / Accuracy= 0.42553192377090454 /Loss= 1.1250361204147339

In [ ]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [ ]:
[(i,i%2) for i in range(0,10)]

In [ ]:
1%2

In [ ]:
2%2

In [ ]:
3%2

In [ ]:
4%2